# Evaluation with ImageNet excluding error data

## 1. Download Imagenet dataset  
First, please register as an ImageNet user [here](https://image-net.org/signup.php).  
After approved, please download 2012 `validation images(all task)` in the same directory.  

## 2. Expand ImageNet into a folder structure with prepro_imagenet.sh

In [ ]:
!sh prepare.sh

## 3. Install Adansons Base with `pip`

In [ ]:
!pip install git+https://github.com/adansons/base

## 4. Create new project
You can create new project via `base new` command.  
Here, we will create a project named imagenet

In [1]:
!base new imagenet

Your Project UID
----------------
b92973bc75e13337b702

save Project UID in local file (~/.base/projects)


## 5. Import data files
Next, add ImageNet to the `imagenet` project.  

You have to specify your dataset directory and file extension via `--directory` and `--extension` option.  
Also, you need to specify the parsing rule via `--parse` option; it automatically parses and links dataset information in the path name according to given parsing rule.

In [4]:
# This could take time depending on the the data amount
# It could take around 3 mins on the MNIST dataset size (= 50000 files)
!base import imagenet --directory imagenet --extension JPEG --parse "{dataType}/{label}/{id}.JPEG"

Check datafiles...
found 50000 files with JPEG extension.
50000/50000 files uploaded.                        
Success!


## 6. Import external metadata files
Import `error_data.csv`, which is the aggregate of ImagenNet error data.  
If you get an error message `Failed to join the table`, this processing continues intenally. (It takes about 5 minutes)  
So, you should run `project.attrs` to check that the processing have finished.

In [5]:
from base import Project
project = Project('imagenet')

# Specify join rules with project tables like `{"New table key": "Exist table key"}
# if you have new key, replace "Exist table key" with None
join_rule = {
    "id": "id",
    "originalLabel": None,
    "correction": None
}
# add metafile
project.add_metafile(
    file_path=["error_data.csv"],
    join_rule=join_rule,
    auto=True
)

1 tables found! (errordata.csv)

1 table joining rule was estimated!

Below table joining rule will be applied...


Rule no.1

	key 'id'	->	connected to 'id' key on exist table
	key 'originalLabel'	->	newly added
	key 'correction'	->	newly added

1 tables will be applied
Table 1 sample record:
	{'id': 'ILSVRC2012_val_00023277', 'originalLabel': 582, 'correction': '930,963,868,923,813,415'}


Exception: Failed to join the tables

In [12]:
# Check to see if the tables have been joined
project = Project('imagenet')
project.attrs

{'id': {'LowerValue': 'ILSVRC2012_val_00000001',
  'UpperValue': 'ILSVRC2012_val_00050000',
  'ValueType': 'str',
  'RecordedCount': 49971},
 'correction': {'LowerValue': '0',
  'UpperValue': '-1',
  'ValueType': 'str',
  'RecordedCount': 14256},
 'label': {'LowerValue': 'n01440764',
  'UpperValue': 'n15075141',
  'ValueType': 'str',
  'RecordedCount': 49971},
 'dataType': {'LowerValue': 'val',
  'UpperValue': 'val',
  'ValueType': 'str',
  'RecordedCount': 49971},
 'originalLabel': {'LowerValue': '0',
  'UpperValue': '999',
  'ValueType': 'int',
  'RecordedCount': 14256}}

## 7. Define Dataset class & preprocess function

In [ ]:
!pip install torch torchvision pillow tqdm

In [13]:
from base import Project, Dataset
from base.files import Files

import torch
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.models as models
from tqdm import tqdm
import json
from typing import Callable, Optional, Tuple
from PIL import Image

In [14]:
# Dataset excluding error data
class ImageNetDataset(Dataset):
    def __init__(
        self,
        files: Files,
        target_key: str,
        model_name: str,
        transform: Optional[Callable] = None,
    ) -> None:

        super().__init__(files, target_key, transform)
        self.external_filter()
        self.y = sorted(set([getattr(i, self.target_key) for i in files]))
        self.convert_dict = {i: j for j, i in enumerate(self.y)}
        self.model_name = model_name

    def __getitem__(self, idx: int) -> Tuple:
        path = self.files_path[idx][1]
        data = self.transform(path, self.model_name)
        label = self.files_path[idx][0].get(self.target_key)
        label = self.convert_dict[label]
        return data, label
    
    def external_filter(self):
        self.files_path = [[record, path] for record, path in zip(self.files.items, self.paths) if not record.get('correction', False)]
    
    def __len__(self):
        return len(self.files_path)


# Functions for image preprocessing
def preprocess_image(image_path, model_name):
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )
    # convert gray to rgb if needed
    image = Image.open(image_path)
    if image.mode != "RGB":
        image = image.convert("RGB")

    sizes = {
        "convnext_large": (232, 224),
        "efficientnet_b7": (633, 600),
        "convnext_base": (232, 224),
        "convnext_small": (230, 224),
        "efficientnet_b5": (489, 456),
        "vit_b_16": (256, 224),
        "regnet_x_32gf": (256, 224),
        "efficientnet_b3": (320, 300),
        "convnext_tiny": (236, 224),
        "efficientnet_b4": (384, 380),
    }
    # last two letters of model_name
    if "efficientnet" in model_name:
        interpolation = transforms.InterpolationMode.BICUBIC
    else:
        interpolation = transforms.InterpolationMode.BILINEAR
    resize_size = sizes[model_name][0]
    crop_size = sizes[model_name][1]
    image = transforms.Resize(resize_size, interpolation=interpolation)(image)
    image = transforms.CenterCrop(crop_size)(image)
    image = transforms.ToTensor()(image)
    image = normalize(image)
    return image

## 8. Evaluation

In [15]:
def eval(model_funcs, val_loader):
    res = {}
    for model_name, model_func in model_funcs.items():
        model = model_func(pretrained=True)
        model.eval()
        model.to("cuda")
        accs = {}
        bar = tqdm(total=len(val_loader))
        bar.set_description(f"Evaluating {model_name}...")
        i = 0
        with torch.no_grad():
            for i, (images, target) in enumerate(val_loader):
                output = model(images.to("cuda"))
                acc = accuracy(output, target.to("cuda"))
                for k, v in acc.items():
                    if k not in accs:
                        accs[k] = torch.tensor([]).to("cuda")
                    accs[k] = torch.cat((accs[k], v))
                bar.update(1)

        for k, v in accs.items():
            print(f"{model_name} top{k} accuracy: {v.mean().item():.2f}%")
            res[model_name + "-" + str(k)] = {"history": v, "mean": v.mean()}

    return res

def accuracy(output, target, topk=(1, 5)):

    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(
            maxk, 1, True, True
        )  # pred : indices. topk : get topk tensors along dim = 1 (within 1000 classes)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        topk_accs = {}
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            topk_accs[k] = correct_k.mul_(100 / batch_size)

        return topk_accs

In [16]:
batch_size = 32

model_funcs = {
    "convnext_large": models.convnext_large,
    # "efficientnet_b7": models.efficientnet_b7,
    # "convnext_base": models.convnext_base,
    # "convnext_small": models.convnext_small,
    # "efficientnet_b5": models.efficientnet_b5,
    # "vit_b_16": models.vit_b_16,
    # "regnet_x_32gf": models.regnet_x_32gf,
    # "efficientnet_b3": models.efficientnet_b3,
    # "convnext_tiny": models.convnext_tiny,
    # "efficientnet_b4": models.efficientnet_b4,
}

# change val_files to your own experiment
val_files = Project("imagenet").files(query=["correction is not None"])
# in order to get the correct label, you need to use the all files
all_files = Project("imagenet").files()

# Start evaluation
res_py = {}
for model_name, model_func in model_funcs.items():
    val_dataset = ImageNetDataset(
        files=all_files,
        target_key="label",
        transform=preprocess_image,
        model_name=model_name,
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size
    )

    res = eval({model_name: model_func}, val_loader)
    # cast data to python list
    for k, v in res.items():
        res_py[k] = {"mean": v["mean"].item()}

# Save evaluation result
with open("imagenet_acc.json", "w") as f:
        json.dump(res_py, f)


Evaluating convnext_large...: 100%|█████████▉| 1116/1117 [03:56<00:00,  4.70it/s]

convnext_large top1 accuracy: 95.57%
convnext_large top5 accuracy: 99.30%


Evaluating convnext_large...: 100%|██████████| 1117/1117 [03:56<00:00,  4.71it/s]
